In [14]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import yfinance as yf

In [61]:
import datetime

# Set the ticker symbol and define the time range
ticker_symbol = 'GE'
end_date = datetime.datetime.now()  # current date and time
start_date = end_date - datetime.timedelta(days=5)

# Fetch intraday minute data using yfinance
msft_data = yf.download(ticker_symbol, start=start_date, end=end_date, interval='1m')
msft_data.reset_index(inplace=True)
# Display the first few rows of the DataFrame
msft_data

[*********************100%%**********************]  1 of 1 completed


,Datetime,Open,High,Low,Close,Adj Close,Volume
0,2023-12-11 09:30:00-05:00,120.589996,120.839996,120.480003,120.739998,120.739998,80800
1,2023-12-11 09:31:00-05:00,120.510002,120.617798,120.510002,120.550003,120.550003,3078
2,2023-12-11 09:32:00-05:00,120.550003,120.620003,120.525002,120.570000,120.570000,3740
3,2023-12-11 09:33:00-05:00,120.574997,120.650002,120.510002,120.650002,120.650002,4087
4,2023-12-11 09:34:00-05:00,120.639999,120.720802,120.570000,120.660004,120.660004,2388
...,...,...,...,...,...,...,...
1557,2023-12-15 09:34:00-05:00,122.029999,122.505898,121.830002,122.290100,122.290100,26726
1558,2023-12-15 09:35:00-05:00,122.480003,122.771301,122.449997,122.580002,122.580002,40497
1559,2023-12-15 09:36:00-05:00,122.510002,122.614998,122.489998,122.565002,122.565002,15517
1560,2023-12-15 09:37:00-05:00,122.555000,122.555000,122.209999,122.239502,122.239502,19713


In [62]:
nan_check = msft_data.isnull().sum()
print("NaN Check:")
print(nan_check)

NaN Check:
Datetime     0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64


In [63]:
def pivotid(df1, l, n1, n2,):
    if l -n1 < 0 or l + n2 >= len(df1):
        return 0
    pividlow = 1
    pividhigh = 1
    for i in range(l - n1, l+n2+1):
        if (df1.Low[l] > df1.Low[i]):
            pividlow = 0
        if (df1.High[l] < df1.High[i]):
            pividhigh = 0
        if pividhigh and pividlow:
            return 3
        if pividlow:
            return 1
        if pividhigh:
            return 2
        else:
            return 0
msft_data['Pivot'] = msft_data.apply(lambda x: pivotid(msft_data, x.name, 3,3), axis=1)        

In [64]:
import numpy as np
def pointpos(x):
    if x['Pivot'] == 1:
        return x['Low']-1e-3
    elif x['Pivot'] == 2:
        return x['High']+1e-3
    else:
        return np.nan
msft_data['pointpos'] = msft_data.apply(lambda row: pointpos(row), axis=1)

In [65]:
import plotly.graph_objects as go 
from plotly.subplots import make_subplots
from datetime import datetime
dfpl = msft_data
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                                     open = dfpl['Open'],
                                     high = dfpl['High'],
                                     low = dfpl['Low'],
                                     close = dfpl['Close'])])
fig.add_scatter(x=dfpl.index, y = dfpl['pointpos'], mode='markers',
                marker=dict(size=5, color = 'MediumPurple'), 
                name = 'Pivot')
fig.show()

In [66]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress

backcandles = 10
candleid = 1000
maxim = np.array([])
minim = np.array([])
xxmin = np.array([])
xxmax = np.array([])
for i in range(candleid - backcandles, candleid+1):
    if msft_data.iloc[i].Pivot == 1:
        minim = np.append(minim, msft_data.iloc[i].Low)
        xxmin = np.append(xxmin, i)
    if msft_data.iloc[i].Pivot == 2:
        maxim = np.append(maxim, msft_data.iloc[i].High)
        xxmax = np.append(xxmax, i)
slmin, intercmin, rmin, pmin, semin = linregress(xxmin, minim)
slmax, intercmax, rmax, pmax, semax = linregress(xxmax, maxim)
print(rmin, rmax)
dfpl = msft_data[candleid-backcandles-10:candleid+backcandles+10]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                                     open = dfpl['Open'],
                                     high = dfpl['High'],
                                     low = dfpl['Low'],
                                     close = dfpl['Close'])])
fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=4, color="MediumPurple"), name='Pivot')

xxmin = np.append(xxmin, xxmin[-1]+15)
xxmax = np.append(xxmax, xxmax[-1]+15)
fig.add_trace(go.Scatter(x=xxmin, y=slmin*xxmin + intercmin, mode='lines', name='min slope'))
fig.add_trace(go.Scatter(x=xxmax, y=slmax*xxmax + intercmax, mode='lines', name='max slope'))
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

-0.9347902644175897 0.907816943464356


In [69]:
for candleid in range(500, len(msft_data)):
    maxim = np.array([])
    minim = np.array([])
    xxmin = np.array([])
    xxmax = np.array([])
    
    for i in range(candleid - backcandles, candleid + 1):
        if msft_data.iloc[i].Pivot == 1:
            minim = np.append(minim, msft_data.iloc[i].Low)
            xxmin = np.append(xxmin, i)  # could be i instead df.iloc[i].name
        if msft_data.iloc[i].Pivot == 2:
            maxim = np.append(maxim, msft_data.iloc[i].High)
            xxmax = np.append(xxmax, i)  # df.iloc[i].name

    # Check for NaN and infinite values
    if any(np.isnan(xxmin)) or any(np.isnan(minim)) or any(np.isnan(xxmax)) or any(np.isnan(maxim)):
        continue

    if any(np.isinf(xxmin)) or any(np.isinf(minim)) or any(np.isinf(xxmax)) or any(np.isinf(maxim)):
        continue

    if (xxmax.size < 3 and xxmin.size < 3) or xxmax.size == 0 or xxmin.size == 0:
        continue

    slmin, intercmin, rmin, pmin, semin = linregress(xxmin, minim)
    slmax, intercmax, rmax, pmax, semax = linregress(xxmax, maxim)

    if abs(rmax) >= 0.9 and abs(rmin) >= 0.9 and slmin >= 0.0001 and slmax <= -0.0001:
        print("Conditions met at candleid:", candleid)
        print("rmin:", rmin, "rmax:", rmax)
        print("slmin:", slmin, "slmax:", slmax)
        break

    if candleid % 100 == 0:
        print("Processing candleid:", candleid)


Processing candleid: 500


C:\Users\Abhimanyu\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\_stats_mstats_common.py:182: RuntimeWarning:

invalid value encountered in scalar divide

C:\Users\Abhimanyu\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\_stats_mstats_common.py:196: RuntimeWarning:

invalid value encountered in sqrt

C:\Users\Abhimanyu\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\_stats_mstats_common.py:199: RuntimeWarning:

invalid value encountered in scalar divide



Processing candleid: 700
Processing candleid: 800
Processing candleid: 900
Processing candleid: 1000
Processing candleid: 1100
Processing candleid: 1200
Processing candleid: 1300
Processing candleid: 1400
Processing candleid: 1500
